In [58]:
# installing google chrome for web scraping javascripts embeddings using selenium for kaggle notebook
!wget https://dl.google.com/linux/linux_signing_key.pub
!sudo apt-key add linux_signing_key.pub
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' >> /etc/apt/sources.list.d/google-chrome.list
!sudo apt-get -y update
!sudo apt-get install -y google-chrome-stable

--2023-01-06 22:20:09--  https://dl.google.com/linux/linux_signing_key.pub
Resolving dl.google.com (dl.google.com)... 142.251.107.93, 142.251.107.91, 142.251.107.136, ...
Connecting to dl.google.com (dl.google.com)|142.251.107.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12415 (12K) [application/octet-stream]
Saving to: ‘linux_signing_key.pub’

linux_signing_key.p 100%[===================>]  12.12K  --.-KB/s    in 0s      

2023-01-06 22:20:09 (53.0 MB/s) - ‘linux_signing_key.pub’ saved [12415/12415]

OK
Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1811 B]
Get:2 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5389 B]    
Get:3 https://packages.cloud.google.com/apt cloud-sdk InRelease [6751 B]       
Get:4 http://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1094 B]
Get:5 http://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [1730 B]
Get:6 https://packages.cloud.google.com/apt cloud-sdk/main amd

In [59]:
# install chromedriver
# !apt-get install -y qq unzip
!wget -O /tmp/chromedriver.zip http://chromedriver.storage.googleapis.com/`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`/chromedriver_linux64.zip
!unzip /tmp/chromedriver.zip chromedriver -d /usr/local/bin/

--2023-01-06 22:21:18--  http://chromedriver.storage.googleapis.com/108.0.5359.71/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 142.250.97.128, 2607:f8b0:400c:c18::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|142.250.97.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7297087 (7.0M) [application/zip]
Saving to: ‘/tmp/chromedriver.zip’

/tmp/chromedriver.z 100%[===================>]   6.96M  --.-KB/s    in 0.04s   

2023-01-06 22:21:19 (167 MB/s) - ‘/tmp/chromedriver.zip’ saved [7297087/7297087]

Archive:  /tmp/chromedriver.zip
  inflating: /usr/local/bin/chromedriver  


In [60]:
# installing selenium
!sudo apt install -y python3-selenium
!pip install selenium==3.141.0 > /dev/null

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 snapd
  squashfs-tools
Suggested packages:
  apparmor-profiles-extra apparmor-utils zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 python3-selenium
  snapd squashfs-tools
0 upgraded, 7 newly installed, 0 to remove and 96 not upgraded.
Need to get 38.4 MB of archives.
After this operation, 173 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 apparmor amd64 2.13.3-7ubuntu5.1 [494 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 liblzo2-2 amd64 2.10-2 [50.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 squashfs-tools amd64 1:4.4-1ubuntu0.3 [117 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 snapd amd

# Importing Libraries

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [61]:
#Importing selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

In [62]:
#Setting up browser to work within kaggle notebook itself
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)

In [8]:
#years selected for scraping using url manipulaiton
selected_years = list(range(1991, 2023))
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

In [9]:
for year in selected_years:
    url = url_start.format(year)
    data = requests.get(url)
    
    with open("{}.hmtl".format(year), "w+") as f:
        f.write(data.text)

In [39]:
mvp_df_list = []
for year in selected_years:
    with open("/kaggle/working/{}.html".format(year)) as f:
        page = f.read()
        soup = BeautifulSoup(page, "html.parser")
        soup.find('tr', class_="over_header").decompose()
        mvp_table = soup.find(id="mvp")
        mvp = pd.read_html(str(mvp_table))[0]
        mvp["Year"] = year
        mvp_df_list.append(mvp)

AttributeError: 'NoneType' object has no attribute 'decompose'

In [53]:
mvp_table = pd.concat(mvp_df_list)

In [71]:
stats_allplayers_url =  "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

for year in selected_years:
    url = stats_allplayers_url.format(year)
    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    time.sleep(2)
    
    html = driver.page_source
    with open("/kaggle/working/players_{}.html".format(year), "w+") as f:
        f.write(html)

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [74]:
players_df_list = []
for year in selected_years:
    with open('/kaggle/working/players_{}.html'.format(year)) as f:
        page = f.read()
        soup = BeautifulSoup(page, "html.parser")
        soup.find('tr', class_="thead").decompose()
        players_table = soup.find(id="per_game_stats")
        player = pd.read_html(str(players_table))[0]
        player["Year"] = year
        players_df_list.append(player)
        print(year)

1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [76]:
players = pd.concat(players_df_list)

In [80]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

for year in selected_years:
    url = team_stats_url.format(year)
    data = requests.get(url)
    
    with open("/kaggle/working/team_{}.html".format(year), "w+") as f:
        f.write(data.text)
    print(year)

1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [1]:
teamdf_list = []

for year in selected_years:
    with open('/kaggle/working/team_{}.html'.format(year)) as f:
        page = f.read()
        soup = BeautifulSoup(page, "html.parser")
        soup.find('tr', class_='thead').decompose()
        soup.find('tr', class_='thead').decompose()
        soup.find('tr', class_='thead').decompose()
        team_table = soup.find(id='divs_standings_E')
        team = pd.read_html(str(team_table))[0]
        team["Year"] = year
        team["Team"] = team["Eastern Conference"]
        del team["Eastern Conference"]
        teamdf_list.append(team)

        soup = BeautifulSoup(page, "html.parser")
        soup.find('tr', class_='thead').decompose()
        soup.find('tr', class_='thead').decompose()
        soup.find('tr', class_='thead').decompose()
        soup.find('tr', class_='thead').decompose()
        team_table = soup.find(id='divs_standings_W')
        team = pd.read_html(str(team_table))[0]
        team["Year"] = year
        team["Team"] = team["Western Conference"]
        del team["Western Conference"]
        teamdf_list.append(team)

NameError: name 'selected_years' is not defined

In [92]:
teams

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,0.683,—,111.5,105.7,5.22,1991,Boston Celtics
1,44,38,0.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers
2,39,43,0.476,17.0,103.1,103.3,-0.43,1991,New York Knicks
3,30,52,0.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,0.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets
...,...,...,...,...,...,...,...,...,...
12,56,26,.683,—,115.6,109.9,5.37,2022,Memphis Grizzlies
13,52,30,.634,4.0,108.0,104.7,3.12,2022,Dallas Mavericks
14,36,46,.439,20.0,109.3,110.3,-0.84,2022,New Orleans Pelicans
15,34,48,.415,22.0,113.2,113.0,0.02,2022,San Antonio Spurs


In [93]:
teams = pd.concat(teamdf_list)

ValueError: No objects to concatenate

In [44]:
mvps = pd.read_excel("/kaggle/input/nba-scrapped-data-1991-2022/mvp_1991_2022.xlsx", sheet_name="Sheet1")
mvps = mvps[["Player", "Year", "Pts Won", "Pts Max", "Share"]]
mvps

,Player,Year,Pts Won,Pts Max,Share
0,Michael Jordan,1991,891,960,0.928
1,Magic Johnson,1991,497,960,0.518
2,David Robinson,1991,476,960,0.496
3,Charles Barkley,1991,222,960,0.231
4,Karl Malone,1991,142,960,0.148
...,...,...,...,...,...
481,Stephen Curry,2022,4,1000,0.004
482,Chris Paul,2022,2,1000,0.002
483,DeMar DeRozan,2022,1,1000,0.001
484,Kevin Durant,2022,1,1000,0.001


In [68]:
players = pd.read_excel("/kaggle/input/nba-scrapped-data-1991-2022/players_1991_2022.xlsx", sheet_name="Sheet1")
drop_columns = ["Unnamed: 0", "Rk"]
players.drop(columns=drop_columns, inplace=True)
players["Player"] = players["Player"].str.replace("*", "", regex=False)

In [74]:
def single_row(df):
    if df.shape[0]==1:
        return df
    else:
        row = df[df["Tm"]=="TOT"]
        row["Tm"] = df.iloc[-1,:]["Tm"]
        return row

In [75]:
players = players.groupby(["Player", "Year"], as_index=False).apply(single_row)
players.reset_index(drop=True, inplace=True)

In [76]:
players

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,.476,...,2.5,3.8,6.3,0.9,0.7,0.3,1.2,1.4,9.1,1991
1,A.C. Green,PF,28,LAL,82,53,35.4,4.7,9.8,.476,...,3.7,5.6,9.3,1.4,1.1,0.4,1.4,1.7,13.6,1992
2,A.C. Green,PF,29,LAL,82,55,34.4,4.6,8.6,.537,...,3.5,5.2,8.7,1.4,1.1,0.5,1.4,1.8,12.8,1993
3,A.C. Green,PF,30,PHO,82,55,34.5,5.7,11.3,.502,...,3.4,5.8,9.2,1.7,0.9,0.5,1.2,1.7,14.7,1994
4,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,.504,...,2.4,5.8,8.2,1.5,0.7,0.4,1.4,1.8,11.2,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14692,Željko Rebrača,C,29,DET,74,4,15.9,2.6,5.1,.505,...,1.1,2.8,3.9,0.5,0.4,1.0,1.1,2.6,6.9,2002
14693,Željko Rebrača,C,30,DET,30,12,16.3,2.7,4.8,.552,...,0.9,2.2,3.1,0.3,0.2,0.6,1.0,2.6,6.6,2003
14694,Željko Rebrača,C,31,ATL,24,2,11.4,1.4,3.2,.442,...,1.0,1.5,2.4,0.3,0.2,0.5,0.7,2.2,3.8,2004
14695,Željko Rebrača,C,32,LAC,58,2,16.0,2.3,4.0,.568,...,0.8,2.3,3.2,0.4,0.2,0.7,0.8,2.2,5.8,2005


In [77]:
merged_players_mvp = players.merge(mvps, how="outer", on=["Player","Year"])
merged_players_mvp[["Pts Won","Pts Max","Share"]] = merged_players_mvp[["Pts Won","Pts Max","Share"]].fillna(0)

In [78]:
merged_players_mvp

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,AST,STL,BLK,TOV,PF,PTS,Year,Pts Won,Pts Max,Share
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,.476,...,0.9,0.7,0.3,1.2,1.4,9.1,1991,0.0,0.0,0.0
1,A.C. Green,PF,28,LAL,82,53,35.4,4.7,9.8,.476,...,1.4,1.1,0.4,1.4,1.7,13.6,1992,0.0,0.0,0.0
2,A.C. Green,PF,29,LAL,82,55,34.4,4.6,8.6,.537,...,1.4,1.1,0.5,1.4,1.8,12.8,1993,0.0,0.0,0.0
3,A.C. Green,PF,30,PHO,82,55,34.5,5.7,11.3,.502,...,1.7,0.9,0.5,1.2,1.7,14.7,1994,0.0,0.0,0.0
4,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,.504,...,1.5,0.7,0.4,1.4,1.8,11.2,1995,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14692,Željko Rebrača,C,29,DET,74,4,15.9,2.6,5.1,.505,...,0.5,0.4,1.0,1.1,2.6,6.9,2002,0.0,0.0,0.0
14693,Željko Rebrača,C,30,DET,30,12,16.3,2.7,4.8,.552,...,0.3,0.2,0.6,1.0,2.6,6.6,2003,0.0,0.0,0.0
14694,Željko Rebrača,C,31,ATL,24,2,11.4,1.4,3.2,.442,...,0.3,0.2,0.5,0.7,2.2,3.8,2004,0.0,0.0,0.0
14695,Željko Rebrača,C,32,LAC,58,2,16.0,2.3,4.0,.568,...,0.4,0.2,0.7,0.8,2.2,5.8,2005,0.0,0.0,0.0


In [94]:
teams = pd.read_excel('/kaggle/input/nba-scrapped-data-1991-2022/team_1991_2022.xlsx', sheet_name="Sheet1", index_col = "Unnamed: 0")
teams["Team"] = teams["Team"].str.replace("*", "", regex=False)
teams = teams[~teams["Team"].str.contains("Division")]

In [95]:
teams

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,0.683,—,111.5,105.7,5.22,1991,Boston Celtics
1,44,38,0.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers
2,39,43,0.476,17.0,103.1,103.3,-0.43,1991,New York Knicks
3,30,52,0.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,0.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets
...,...,...,...,...,...,...,...,...,...
12,56,26,.683,—,115.6,109.9,5.37,2022,Memphis Grizzlies
13,52,30,.634,4.0,108.0,104.7,3.12,2022,Dallas Mavericks
14,36,46,.439,20.0,109.3,110.3,-0.84,2022,New Orleans Pelicans
15,34,48,.415,22.0,113.2,113.0,0.02,2022,San Antonio Spurs


In [82]:
team_nicknames = pd.read_csv("/kaggle/input/nba-scrapped-data-1991-2022/team_nicknames.txt", delimiter=",")

name_dict_map = {}

for i in range(team_nicknames.shape[0]):
    name_dict_map[team_nicknames.iloc[i,0]] = team_nicknames.iloc[i,1]

In [83]:
merged_players_mvp["Team"] = merged_players_mvp["Tm"].map(name_dict_map)
merged_players_mvp

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,STL,BLK,TOV,PF,PTS,Year,Pts Won,Pts Max,Share,Team
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,.476,...,0.7,0.3,1.2,1.4,9.1,1991,0.0,0.0,0.0,Los Angeles Lakers
1,A.C. Green,PF,28,LAL,82,53,35.4,4.7,9.8,.476,...,1.1,0.4,1.4,1.7,13.6,1992,0.0,0.0,0.0,Los Angeles Lakers
2,A.C. Green,PF,29,LAL,82,55,34.4,4.6,8.6,.537,...,1.1,0.5,1.4,1.8,12.8,1993,0.0,0.0,0.0,Los Angeles Lakers
3,A.C. Green,PF,30,PHO,82,55,34.5,5.7,11.3,.502,...,0.9,0.5,1.2,1.7,14.7,1994,0.0,0.0,0.0,Phoenix Suns
4,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,.504,...,0.7,0.4,1.4,1.8,11.2,1995,0.0,0.0,0.0,Phoenix Suns
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14692,Željko Rebrača,C,29,DET,74,4,15.9,2.6,5.1,.505,...,0.4,1.0,1.1,2.6,6.9,2002,0.0,0.0,0.0,Detroit Pistons
14693,Željko Rebrača,C,30,DET,30,12,16.3,2.7,4.8,.552,...,0.2,0.6,1.0,2.6,6.6,2003,0.0,0.0,0.0,Detroit Pistons
14694,Željko Rebrača,C,31,ATL,24,2,11.4,1.4,3.2,.442,...,0.2,0.5,0.7,2.2,3.8,2004,0.0,0.0,0.0,Atlanta Hawks
14695,Željko Rebrača,C,32,LAC,58,2,16.0,2.3,4.0,.568,...,0.2,0.7,0.8,2.2,5.8,2005,0.0,0.0,0.0,Los Angeles Clippers


In [109]:
#joining data and improving data quality
stats = merged_players_mvp.merge(teams, how='outer', on=["Team", "Year"])
stats["GB"] = stats["GB"].str.replace("—", "0")
stats = stats.apply(pd.to_numeric, errors='ignore')
stats.fillna(0, inplace=True)

In [112]:
#downloading the data to excel
stats.to_excel("full_stats_1991_2022.xlsx")